# PASO 1 - Obtención de datos de Google Trends (Automatizado con Pytrends)

Este cuaderno tiene como objetivo recolectar datos de tendencias de búsqueda a nivel global o regional de forma **automatizada**.  
Utilizaremos la librería [Pytrends](https://github.com/GeneralMills/pytrends) para interactuar con la API no oficial de Google Trends y descargar los resultados en formato CSV, dejando todo **listo** para el análisis posterior.

---

## 1. Instalación de librerías necesarias

Primero, nos aseguramos de instalar/actualizar Pytrends en caso de no tenerlo.  
Si estás en Google Colab, Jupyter local u otro entorno, ejecuta la siguiente celda.

In [25]:
%%capture
%pip install --upgrade pytrends

## 2. Importación de librerías y configuración inicial

- **Pandas**: Para manipular y guardar los datos en CSV.  
- **Pytrends**: Para conectarnos a Google Trends.  
- Configuramos la zona horaria y el idioma deseado.  

In [26]:
import os
import time
import pandas as pd
from pytrends.request import TrendReq

# =====================================
# Configuración inicial de Pytrends
# =====================================
# hl: lenguaje (por ejemplo, 'en-US', 'es-ES')
# tz: zona horaria (offset en minutos del UTC)
pytrends = TrendReq(hl='en-US', tz=360)

## 3. Definición de parámetros de búsqueda

En esta sección establecemos:
- Lista de **palabras clave** (o tópicos) que queremos investigar.
- Rango de tiempo de interés, por ejemplo 'today 12-m' (últimos 12 meses).
- Ubicación geográfica (geo), que puede ser '' para "Todo el mundo" o un código de país, p.ej. 'US', 'ES', etc.

**Nota:** Para este "PASO 1", haremos una búsqueda inicial y amplia con un conjunto de keywords genéricos para explorar tendencias globales.

In [28]:
# Lista de palabras clave para la exploración
keywords = [
    'shopping',
    'fitness',
    'technology',
    'beauty',
    'health'
]

# Opciones de timeframe y ubicación
timeframe = 'today 12-m'   # Últimos 12 meses
geo = ''                   # '' = Worldwide, también puedes usar 'ES' (España), 'US' (Estados Unidos), etc.

# Carpetas para guardar datos
DATA_FOLDER = '../data'
if not os.path.exists(DATA_FOLDER):
    os.makedirs(DATA_FOLDER)

## 4. Construcción de la "payload" y obtención de datos de "Interest Over Time"

Con Pytrends, cada vez que construimos una *payload*, especificamos las keywords, el rango de tiempo y la región.
Luego, podemos obtener:
- **Interest over time**: los niveles de interés a lo largo del tiempo.
- **Related topics**: temas relacionados que la gente busca junto a nuestras keywords.
- **Related queries**: consultas relacionadas (búsquedas específicas).
- **Interest by region**: para ver la distribución geográfica del interés.

In [29]:
# Paso 4.1: Construir la payload de Pytrends
pytrends.build_payload(
    kw_list=keywords,
    timeframe=timeframe,
    geo=geo
)

# Paso 4.2: Obtener el "Interest Over Time"
interest_over_time_df = pytrends.interest_over_time()

# Revisamos un pequeño vistazo al dataframe
print("Interest over time (primeras 5 filas):")
display(interest_over_time_df.head())

# Guardamos en CSV
iot_csv_path = os.path.join(DATA_FOLDER, 'interest_over_time.csv')
interest_over_time_df.to_csv(iot_csv_path, index=True)
print(f"Archivo guardado: {iot_csv_path}")

Interest over time (primeras 5 filas):


/Users/josephcode/.pyenv/versions/3.10.13/lib/python3.10/site-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


,shopping,fitness,technology,beauty,health,isPartial
date,,,,,,
2023-12-24,42,30,14,41,65,False
2023-12-31,39,38,17,39,84,False
2024-01-07,33,32,19,36,90,False
2024-01-14,32,32,20,35,87,False
2024-01-21,33,31,22,35,90,False


Archivo guardado: ../data/interest_over_time.csv


## 5. Related Topics y Related Queries

Estas funciones proveen información de los temas y búsquedas que Google asocia con nuestras keywords.  
Podremos descubrir tendencias o conceptos emergentes que no habíamos considerado.

In [10]:
# Paso 5.1: Related Topics
# Esto devuelve un diccionario cuyas keys son cada keyword,
# y el contenido son dataframes de "top" y "rising" topics.

# Configurar Pytrends con reintentos y backoff
pytrends = TrendReq(
    hl='en-US',
    tz=360,
    retries=5,
    backoff_factor=0.1
)

DATA_FOLDER = './data'
os.makedirs(DATA_FOLDER, exist_ok=True)

keywords = ['shopping', 'fitness', 'technology']

for kw in keywords:
    # 1. Construir payload
    pytrends.build_payload(
        kw_list=[kw],
        timeframe='today 5-y',
        geo='US'
    )

    # Manejar el caso en que Google no devuelva 'rankedList'
    try:
        related_topics_dict = pytrends.related_topics()
    except IndexError:
        print(f"[WARNING] Google no devolvió datos de 'rankedList' para '{kw}'. Skipping...")
        continue

    # 2. Procesar 'top' y 'rising' solo si existen
    if kw in related_topics_dict:
        # 'top'
        if 'top' in related_topics_dict[kw] and not related_topics_dict[kw]['top'].empty:
            df_top_topics = related_topics_dict[kw]['top']
            df_top_path = os.path.join(DATA_FOLDER, f'related_topics_top_{kw}.csv')
            df_top_topics.to_csv(df_top_path, index=False)
            print(f"Related TOP topics para '{kw}' guardados en: {df_top_path}")
        else:
            print(f"Sin datos de 'top' para '{kw}'")

        # 'rising'
        if 'rising' in related_topics_dict[kw] and not related_topics_dict[kw]['rising'].empty:
            df_rising_topics = related_topics_dict[kw]['rising']
            df_rising_path = os.path.join(DATA_FOLDER, f'related_topics_rising_{kw}.csv')
            df_rising_topics.to_csv(df_rising_path, index=False)
            print(f"Related RISING topics para '{kw}' guardados en: {df_rising_path}")
        else:
            print(f"Sin datos de 'rising' para '{kw}'")
    else:
        print(f"[WARNING] '{kw}' no está en 'related_topics_dict'. No hay datos.")

    # 3. Related Queries
    # (mismo tipo de verificación)
    try:
        related_queries_dict = pytrends.related_queries()
    except IndexError:
        print(f"[WARNING] Google no devolvió queries para '{kw}'. Skipping...")
        continue

    if kw in related_queries_dict:
        if 'top' in related_queries_dict[kw] and not related_queries_dict[kw]['top'].empty:
            df_top_queries = related_queries_dict[kw]['top']
            df_topq_path = os.path.join(DATA_FOLDER, f'related_queries_top_{kw}.csv')
            df_top_queries.to_csv(df_topq_path, index=False)
            print(f"Related TOP queries para '{kw}' guardados en: {df_topq_path}")
        else:
            print(f"Sin datos de 'top queries' para '{kw}'")

        if 'rising' in related_queries_dict[kw] and not related_queries_dict[kw]['rising'].empty:
            df_rising_queries = related_queries_dict[kw]['rising']
            df_risingq_path = os.path.join(DATA_FOLDER, f'related_queries_rising_{kw}.csv')
            df_rising_queries.to_csv(df_risingq_path, index=False)
            print(f"Related RISING queries para '{kw}' guardados en: {df_risingq_path}")
        else:
            print(f"Sin datos de 'rising queries' para '{kw}'")
    else:
        print(f"[WARNING] '{kw}' no está en 'related_queries_dict'. No hay datos.")

    time.sleep(5)  # Pausa para evitar bloqueos

[WARNING] Google no devolvió datos de 'rankedList' para 'shopping'. Skipping...
[WARNING] Google no devolvió datos de 'rankedList' para 'fitness'. Skipping...
[WARNING] Google no devolvió datos de 'rankedList' para 'technology'. Skipping...


## **Ampliación: Identificar países con mayor interés y profundizar en Related Topics**

In [24]:
import os
import time
import pandas as pd
from pytrends.request import TrendReq

# ======================================================
# 1. CONFIGURACIÓN PYTRENDS “AL LÍMITE”
# ======================================================
pytrends = TrendReq(
    hl='en-US',
    tz=360,
    retries=5,
    backoff_factor=0.1
)

# Carpeta para CSV
DATA_FOLDER = 'data'
os.makedirs(DATA_FOLDER, exist_ok=True)

# Palabras clave “complicadas”
keywords = [
    'laptop', 'airpods', 'headphones', 'wireless earbuds', 'ipad', 'fire stick', 'fitbit', 'tv', 'air fryer',
    'bluetooth headphones', 'roku', 'external hard drive', 'micro sd card', 'gaming chair', 'apple watch',
    'monitor', 'earbuds', 'paper towels', 'desk', 'office chair', 'ring doorbell', 'lego', 'wireless mouse',
    'mouse pad', 'iphone charger', 'echo dot', 'nintendo switch games', 'keyboard', 'bluetooth speakers',
    'iphone', 'coffee', 'aa batteries', 'mouse', 'shower curtain', 'microwave', 'protein powder', 'shoe rack',
    'iphone 11 case', 'yoga mat', 'gaming pc', 'usb hub', 'fan', 'portable charger', 'xbox one', 'gift cards',
    'electric toothbrush', 'rice cooker', 'mini fridge', 'umbrella', 'batteries', 'pens', 'extension cord',
    'toothpaste', 'desk chair', 'blood pressure monitor', 'coffee maker', 'dehumidifier', 'dog bed', 'magnesium',
    'dog toys', 'vitamin d', 'ear buds', 'dog food', 'shower head', 'notebook', 'water', 'womens tops',
    'led lights', 'wifi extender', 'baby wipes', 'crocs', 'yeti', 'twin mattress', 'laundry basket',
    'shower caddy', 'apple watch band', 'fish oil', 'hangers', 'vitamin c', 'macbook', 'paper plates',
    'amazon gift card', 'snacks', 'pillows', 'ear plugs', 'hand soap', 'creatine', 'nintendo switch controller',
    'queen bed frame', 'dresses', 'laptop stand', 'iphone 12 case', 'water bottle', 'face mask', 'sunscreen',
    'hair dryer', 'makeup brushes', 'lip balm', 'nail polish', 'eye cream', 'foundation', 'shampoo',
    'conditioner', 'body lotion', 'hand sanitizer', 'toilet paper', 'dish soap', 'laundry detergent',
    'fabric softener', 'trash bags', 'aluminum foil', 'plastic wrap', 'paper towels', 'napkins',
    'disinfecting wipes', 'bleach', 'all-purpose cleaner', 'glass cleaner', 'sponges', 'mop', 'broom',
    'vacuum cleaner', 'air freshener', 'candles', 'light bulbs', 'batteries', 'extension cord', 'power strip',
    'surge protector', 'smoke detector', 'carbon monoxide detector', 'fire extinguisher', 'first aid kit',
    'thermometer', 'humidifier', 'space heater', 'fan', 'air purifier', 'water filter', 'coffee maker',
    'blender', 'toaster', 'microwave', 'slow cooker', 'pressure cooker', 'rice cooker', 'electric kettle',
    'stand mixer', 'hand mixer', 'food processor', 'juicer', 'ice cream maker', 'bread machine', 'waffle maker',
    'grill', 'griddle', 'frying pan', 'saucepan', 'stockpot', 'baking sheet', 'cake pan', 'muffin tin', 'pie dish',
    'cutting board', 'knife set', 'measuring cups', 'measuring spoons', 'mixing bowls', 'colander', 'can opener',
    'peeler', 'whisk', 'spatula', 'tongs', 'ladle', 'slotted spoon', 'pizza cutter', 'kitchen timer', 'oven mitts',
    'pot holders', 'apron', 'dish rack', 'dish towels', 'placemats', 'coasters', 'tablecloth', 'napkin rings',
    'salt and pepper shakers', 'oil dispenser', 'vinegar dispenser', 'sugar bowl', 'butter dish', 'bread box',
    'fruit basket', 'cookie jar', 'paper towel holder', 'soap dispenser', 'sink caddy', 'trash can', 'recycling bin',
    'step stool', 'ladder', 'tool set', 'flashlight', 'tape measure', 'level', 'screwdriver set', 'hammer', 'pliers',
    'wrench set', 'drill', 'saw', 'utility knife', 'duct tape', 'super glue', 'nails', 'screws', 'anchors', 'hooks',
    'picture hangers', 'bicycle', 'helmet', 'bike lock', 'bike pump', 'water bottle holder', 'cycling gloves',
    'cycling shorts', 'cycling jersey', 'running shoes', 'running socks', 'fitness tracker', 'yoga mat', 'dumbbells',
    'resistance bands', 'jump rope', 'treadmill', 'exercise bike', 'rowing machine', 'elliptical machine', 'weight bench',
    'kettlebell', 'medicine ball', 'foam roller', 'massage gun', 'protein powder', 'pre-workout', 'post-workout',
    'multivitamin', 'fish oil', 'vitamin D', 'vitamin C', 'magnesium', 'zinc', 'probiotics', 'collagen', 'greens powder',
    'meal replacement', 'snack bars', 'granola bars', 'trail mix', 'dried fruit', 'nuts', 'seeds', 'jerky', 'popcorn',
    'chips', 'crackers', 'cookies', 'candy', 'chocolate', 'gum', 'mints', 'coffee', 'tea', 'hot chocolate', 'energy drinks',
    'sports drinks', 'water bottles', 'travel mugs', 'thermos', 'cooler', 'lunch box', 'backpack', 'duffel bag',
    'suitcase', 'luggage set', 'passport holder', 'luggage tags', 'travel pillow', 'eye mask', 'ear plugs',
    'compression socks', 'packing cubes', 'toiletry bag', 'hanging toiletry bag', 'makeup bag', 'jewelry organizer',
    'shoe bag', 'laundry bag', 'garment bag', 'umbrella', 'raincoat', 'poncho', 'sunscreen', 'sunglasses',
    'hat', 'scarf', 'gloves', 'beanie', 'belt', 'wallet', 'purse', 'handbag', 'tote bag', 'crossbody bag',
    'clutch', 'wristlet', 'watch', 'bracelet', 'necklace', 'earrings', 'ring', 'anklet', 'brooch', 'hair clips',
    'hair ties', 'headband'
]

# GProp: distintas propiedades de búsqueda
#  - '' (búsqueda web), 'images', 'news', 'youtube', 'froogle' (Google Shopping)
GPROPS = ['', 'images', 'news', 'youtube', 'froogle']

# Categoría general (0 = todas)
CATEGORY = 0

# Único timeframe “ALL” (2004–actual)
TIMEFRAME = 'all'

# Para filtrar *sólo* países con códigos en tu diccionario
country_code_map = {
    'Afghanistan': 'AF',
    'Albania': 'AL',
    'Algeria': 'DZ',
    'Andorra': 'AD',
    'Angola': 'AO',
    'Antigua & Barbuda': 'AG',
    'Argentina': 'AR',
    'Armenia': 'AM',
    'Australia': 'AU',
    'Austria': 'AT',
    'Azerbaijan': 'AZ',
    'Bahamas': 'BS',
    'Bahrain': 'BH',
    'Bangladesh': 'BD',
    'Barbados': 'BB',
    'Belarus': 'BY',
    'Belgium': 'BE',
    'Belize': 'BZ',
    'Benin': 'BJ',
    'Bhutan': 'BT',
    'Bolivia': 'BO',
    'Bosnia & Herzegovina': 'BA',
    'Botswana': 'BW',
    'Brazil': 'BR',
    'Brunei': 'BN',
    'Bulgaria': 'BG',
    'Burkina Faso': 'BF',
    'Burundi': 'BI',
    'Cabo Verde': 'CV',
    'Cambodia': 'KH',
    'Cameroon': 'CM',
    'Canada': 'CA',
    'Central African Republic': 'CF',
    'Chad': 'TD',
    'Chile': 'CL',
    'China': 'CN',
    'Colombia': 'CO',
    'Comoros': 'KM',
    'Congo (Congo-Brazzaville)': 'CG',
    'Congo (DRC)': 'CD',
    'Costa Rica': 'CR',
    'Cote d’Ivoire': 'CI',
    'Croatia': 'HR',
    'Cuba': 'CU',
    'Cyprus': 'CY',
    'Czechia': 'CZ',
    'Denmark': 'DK',
    'Djibouti': 'DJ',
    'Dominica': 'DM',
    'Dominican Republic': 'DO',
    'Ecuador': 'EC',
    'Egypt': 'EG',
    'El Salvador': 'SV',
    'Equatorial Guinea': 'GQ',
    'Eritrea': 'ER',
    'Estonia': 'EE',
    'Eswatini': 'SZ',
    'Ethiopia': 'ET',
    'Fiji': 'FJ',
    'Finland': 'FI',
    'France': 'FR',
    'Gabon': 'GA',
    'Gambia': 'GM',
    'Georgia': 'GE',
    'Germany': 'DE',
    'Ghana': 'GH',
    'Greece': 'GR',
    'Grenada': 'GD',
    'Guatemala': 'GT',
    'Guinea': 'GN',
    'Guinea-Bissau': 'GW',
    'Guyana': 'GY',
    'Haiti': 'HT',
    'Honduras': 'HN',
    'Hungary': 'HU',
    'Iceland': 'IS',
    'India': 'IN',
    'Indonesia': 'ID',
    'Iran': 'IR',
    'Iraq': 'IQ',
    'Ireland': 'IE',
    'Israel': 'IL',
    'Italy': 'IT',
    'Jamaica': 'JM',
    'Japan': 'JP',
    'Jordan': 'JO',
    'Kazakhstan': 'KZ',
    'Kenya': 'KE',
    'Kiribati': 'KI',
    'Kuwait': 'KW',
    'Kyrgyzstan': 'KG',
    'Laos': 'LA',
    'Latvia': 'LV',
    'Lebanon': 'LB',
    'Lesotho': 'LS',
    'Liberia': 'LR',
    'Libya': 'LY',
    'Liechtenstein': 'LI',
    'Lithuania': 'LT',
    'Luxembourg': 'LU',
    'Madagascar': 'MG',
    'Malawi': 'MW',
    'Malaysia': 'MY',
    'Maldives': 'MV',
    'Mali': 'ML',
    'Malta': 'MT',
    'Marshall Islands': 'MH',
    'Mauritania': 'MR',
    'Mauritius': 'MU',
    'Mexico': 'MX',
    'Micronesia': 'FM',
    'Moldova': 'MD',
    'Monaco': 'MC',
    'Mongolia': 'MN',
    'Montenegro': 'ME',
    'Morocco': 'MA',
    'Mozambique': 'MZ',
    'Myanmar': 'MM',
    'Namibia': 'NA',
    'Nauru': 'NR',
    'Nepal': 'NP',
    'Netherlands': 'NL',
    'New Zealand': 'NZ',
    'Nicaragua': 'NI',
    'Niger': 'NE',
    'Nigeria': 'NG',
    'North Korea': 'KP',
    'North Macedonia': 'MK',
    'Norway': 'NO',
    'Oman': 'OM',
    'Pakistan': 'PK',
    'Palau': 'PW',
    'Palestine State': 'PS',
    'Panama': 'PA',
    'Papua New Guinea': 'PG',
    'Paraguay': 'PY',
    'Peru': 'PE',
    'Philippines': 'PH',
    'Poland': 'PL',
    'Portugal': 'PT',
    'Qatar': 'QA',
    'Romania': 'RO',
    'Russia': 'RU',
    'Rwanda': 'RW',
    'Saint Kitts & Nevis': 'KN',
    'Saint Lucia': 'LC',
    'Saint Vincent & the Grenadines': 'VC',
    'Samoa': 'WS',
    'San Marino': 'SM',
    'Sao Tome & Principe': 'ST',
    'Saudi Arabia': 'SA',
    'Senegal': 'SN',
    'Serbia': 'RS',
    'Seychelles': 'SC',
    'Sierra Leone': 'SL',
    'Singapore': 'SG',
    'Slovakia': 'SK',
    'Slovenia': 'SI',
    'Solomon Islands': 'SB',
    'Somalia': 'SO',
    'South Africa': 'ZA',
    'South Korea': 'KR',
    'South Sudan': 'SS',
    'Spain': 'ES',
    'Sri Lanka': 'LK',
    'Sudan': 'SD',
    'Suriname': 'SR',
    'Sweden': 'SE',
    'Switzerland': 'CH',
    'Syria': 'SY',
    'Tajikistan': 'TJ',
    'Tanzania': 'TZ',
    'Thailand': 'TH',
    'Timor-Leste': 'TL',
    'Togo': 'TG',
    'Tonga': 'TO',
    'Trinidad & Tobago': 'TT',
    'Tunisia': 'TN',
    'Turkey': 'TR',
    'Turkmenistan': 'TM',
    'Tuvalu': 'TV',
    'Uganda': 'UG',
    'Ukraine': 'UA',
    'United Arab Emirates': 'AE',
    'United Kingdom': 'GB',
    'United States': 'US',
    'Uruguay': 'UY',
    'Uzbekistan': 'UZ',
    'Vanuatu': 'VU',
    'Vatican City': 'VA',
    'Venezuela': 'VE',
    'Vietnam': 'VN',
    'Yemen': 'YE',
    'Zambia': 'ZM',
    'Zimbabwe': 'ZW',
}

# ======================================================
# 2. OBTENER TOP PAÍSES A NIVEL GLOBAL, USANDO timeframe='all'
# ======================================================
def get_top_countries(keyword, max_countries=10):
    """
    Hace una sola consulta timeframe='all', cat=0, gprop=''
    para obtener interest_by_region a nivel mundial.
    Retorna un Series con top X países y su interés (>0).
    """
    print(f"[GLOBAL SCAN] keyword='{keyword}', timeframe='all', category={CATEGORY}, gprop=''")
    
    # Build payload global
    pytrends.build_payload(
        kw_list=[keyword],
        timeframe=TIMEFRAME,  
        geo='',              
        cat=CATEGORY,        
        gprop=''             # Búsqueda web
    )
    
    # Interés por país (toda la historia)
    df = pytrends.interest_by_region(
        resolution='COUNTRY',
        inc_low_vol=True
    )
    if keyword not in df.columns:
        print(f"[WARNING] No hay columna {keyword} en interest_by_region. Regresando vacío.")
        return pd.Series([], dtype=float)
    
    # Filtrar sólo países de tu mapa (si quieres descartar subregiones)
    valid_countries = set(country_code_map.keys())
    df = df[df.index.isin(valid_countries)]
    
    # Ordenar y tomar los países top
    top_series = df[keyword].sort_values(ascending=False).head(max_countries)
    top_series = top_series[top_series > 0]
    
    return top_series


# ======================================================
# 3. BUSCAR RELATED TOPICS EN MODO “AGRESIVO”
# ======================================================
def analyze_related_topics_aggresive(keyword, country_name, iso_code):
    """
    Intenta varios gprop (web, images, news, youtube, froogle)
    con timeframe='all' y cat=0, para ver si encuentra algo de related topics.
    """
    # Se creará un “historial” de lo que intentamos
    found_something = False
    
    for gprop_mode in GPROPS:
        print(f"  -> Intentando gprop='{gprop_mode}' en {country_name} ({iso_code})")
        
        # Build payload específico
        pytrends.build_payload(
            kw_list=[keyword],
            timeframe=TIMEFRAME,
            geo=iso_code,
            cat=CATEGORY,
            gprop=gprop_mode
        )
        
        try:
            related_dict = pytrends.related_topics()
        except IndexError:
            print(f"  [WARNING] Google no brindó related topics (IndexError) con gprop='{gprop_mode}'")
            continue
        
        if keyword not in related_dict:
            print(f"  [WARNING] No data para '{keyword}' en related_dict con gprop='{gprop_mode}'")
            continue
        
        # Top
        if 'top' in related_dict[keyword] and not related_dict[keyword]['top'].empty:
            df_top = related_dict[keyword]['top']
            csv_top = os.path.join(DATA_FOLDER, f"RELTOP_{keyword}_{iso_code}_{gprop_mode}.csv")
            df_top.to_csv(csv_top, index=False)
            print(f"    [SUCCESS] TOP en {csv_top}  (rows={len(df_top)})")
            found_something = True
        else:
            print("    Sin datos TOP")
        
        # Rising
        if 'rising' in related_dict[keyword] and not related_dict[keyword]['rising'].empty:
            df_rising = related_dict[keyword]['rising']
            csv_rising = os.path.join(DATA_FOLDER, f"RELRISING_{keyword}_{iso_code}_{gprop_mode}.csv")
            df_rising.to_csv(csv_rising, index=False)
            print(f"    [SUCCESS] RISING en {csv_rising}  (rows={len(df_rising)})")
            found_something = True
        else:
            print("    Sin datos RISING")
        
        # Pausa mínima
        time.sleep(3)
    
    return found_something


# ======================================================
# 4. FLUJO PRINCIPAL
# ======================================================
if __name__ == "__main__":
    for kw in keywords:
        print("\n========================================")
        print(f" PROCESANDO KEYWORD: {kw.upper()}")
        print("========================================\n")
        
        # 1) Obtener top países (timeframe='all', cat=0, gprop='')
        top_countries = get_top_countries(kw, max_countries=5)
        if top_countries.empty:
            print(f"[WARNING] Ningún país con interés > 0 para '{kw}' con timeframe='all'")
            continue
        
        print(f"Top {len(top_countries)} países:\n{top_countries}\n")
        
        # 2) Para cada país, intentar related topics en modo “agresivo”
        for country_name in top_countries.index:
            iso_code = country_code_map.get(country_name)
            if not iso_code:
                print(f"[WARNING] No hay mapeo iso para {country_name}. Skipping.")
                continue
            
            print(f"\n[AGGRESIVE SEARCH] '{kw}' en {country_name} (geo='{iso_code}')")
            success = analyze_related_topics_aggresive(kw, country_name, iso_code)
            if not success:
                print("  => No se encontró NADA de related topics ni en web ni en images/news/youtube/froogle.\n")
            
            time.sleep(5)  # Pausa mayor entre países


 PROCESANDO KEYWORD: LAPTOP

[GLOBAL SCAN] keyword='laptop', timeframe='all', category=0, gprop=''
Top 5 países:
geoName
Tuvalu       100
Kiribati      85
Eritrea       80
Indonesia     68
Bhutan        65
Name: laptop, dtype: int64


[AGGRESIVE SEARCH] 'laptop' en Tuvalu (geo='TV')
  -> Intentando gprop='' en Tuvalu (TV)
  [WARNING] Google no brindó related topics (IndexError) con gprop=''
  -> Intentando gprop='images' en Tuvalu (TV)
  [WARNING] Google no brindó related topics (IndexError) con gprop='images'
  -> Intentando gprop='news' en Tuvalu (TV)
  [WARNING] Google no brindó related topics (IndexError) con gprop='news'
  -> Intentando gprop='youtube' en Tuvalu (TV)
  [WARNING] Google no brindó related topics (IndexError) con gprop='youtube'
  -> Intentando gprop='froogle' en Tuvalu (TV)
  [WARNING] Google no brindó related topics (IndexError) con gprop='froogle'
  => No se encontró NADA de related topics ni en web ni en images/news/youtube/froogle.


[AGGRESIVE SEARCH] 'laptop'

# FASE 2: Exploración de la Competencia en E-commerce

En este paso, iniciaremos el **análisis de la competencia** para las categorías o productos que te interesan (por ejemplo, aquellos que identificaste en la Fase 1 a partir de Google Trends). El objetivo es **cuantificar** la competencia presente en plataformas de e-commerce y entender mejor el mercado antes de predecir tendencias de venta.

## **Objetivos de esta Fase**:
1. **Recolectar** datos de e-commerce (por ejemplo, Amazon) sobre los productos relacionados a tus keywords.
2. **Medir** la densidad de la competencia: cuántos vendedores/proveedores hay, sus precios, sus valoraciones, etc.
3. **Empezar** a esbozar un “índice de competencia” que luego podrás integrar en tu *Opportunity Score* (o cualquier métrica final que decidas usar).

## **Enfoque**:
- Utilizaremos **Python** (requests + BeautifulSoup) para hacer una recolección rápida (scraping) de la página de Amazon con la keyword seleccionada.  
- Guardaremos los datos en un archivo CSV para su análisis posterior.  

> **Nota**: Amazon es muy estricto con el scraping; se recomienda **no abusar** de las peticiones y, si es posible, usar un tiempo de espera (*sleep*) entre cada página para evitar bloqueos. Además, revisa sus Términos de Servicio para asegurarte de no infringir políticas.  
> Si prefieres una API oficial, puedes investigar la [Amazon Product Advertising API](https://affiliate-program.amazon.com/) (requiere registro y tokens).

## **Pasos del proceso**:
1. **Definir** la keyword o producto que quieras investigar.
2. **Enviar** una petición a la **URL** de búsqueda de Amazon correspondiente (ej. `https://www.amazon.com/s?k={keyword}`).
3. **Parsear** la respuesta HTML para extraer:
   - Nombre del producto
   - Precio
   - Valoraciones (estrellas, número de reseñas)
   - URL del producto (si es relevante)
4. **Guardar** la información en un CSV (`productos_competencia.csv`).
5. **Revisar** los resultados y ajustar en caso de errores o campos faltantes.

---

In [30]:
# =====================================
# FASE 2: Exploración de la Competencia
# =====================================
# Ejemplo de Scraping en Amazon con requests + BeautifulSoup
# (Ajustar según país/idioma: amazon.com, amazon.es, etc.)

import os
import time
import csv
import requests
from bs4 import BeautifulSoup

# Parámetros iniciales
KEYWORD = "shopping"  # Aquí puedes poner la keyword que quieras (ej. "fitness bands")
BASE_URL = "https://www.amazon.com"  # Cambia a .es, .co.uk, etc., según el mercado
OUTPUT_FILE = "productos_competencia.csv"

# Parámetros de cabecera para "fingir" un navegador
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/109.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
}

def scrape_amazon_competitors(keyword, pages=1):
    """
    Busca la keyword en Amazon y extrae datos básicos de cada producto.
    pages: Número de páginas a recorrer (cuidado con bloqueos).
    Devuelve una lista de diccionarios con los campos.
    """

    all_products = []

    for page in range(1, pages + 1):
        print(f"\n[INFO] Scrape de la página {page} para la keyword '{keyword}'")

        # Ej. URL de búsqueda: https://www.amazon.com/s?k=shopping&page=1
        search_url = f"{BASE_URL}/s?k={keyword}&page={page}"

        resp = requests.get(search_url, headers=HEADERS)
        if resp.status_code != 200:
            print(f"[WARNING] Status code {resp.status_code} en la página {page}. Deteniendo.")
            break

        soup = BeautifulSoup(resp.text, "html.parser")

        # Encontramos contenedores de productos (esto puede cambiar a futuro según Amazon)
        product_containers = soup.find_all("div", {"data-component-type": "s-search-result"})

        # Extraer info de cada contenedor
        for product in product_containers:
            # ID del producto (asin)
            asin = product.get("data-asin", "").strip()
            if not asin:
                continue  # A veces data-asin está vacío en contenedores de publicidad

            # Título
            title_el = product.find("span", class_="a-size-base-plus a-color-base a-text-normal")
            title = title_el.get_text(strip=True) if title_el else "N/A"

            # Link al producto
            link_el = product.find("a", class_="a-link-normal a-text-normal")
            product_url = f"{BASE_URL}{link_el.get('href')}" if link_el else "N/A"

            # Precio (puede variar en Amazon, esto es un ejemplo)
            price_whole = product.select_one("span.a-price span.a-price-whole")
            price_fraction = product.select_one("span.a-price span.a-price-fraction")
            if price_whole and price_fraction:
                price = f"{price_whole.get_text(strip=True)}.{price_fraction.get_text(strip=True)}"
            else:
                price = "N/A"

            # Reseñas (estrellas y número de reseñas)
            rating_el = product.find("span", class_="a-icon-alt")
            rating_text = rating_el.get_text(strip=True) if rating_el else "N/A"
            # Esto suele ser algo como '4.5 out of 5 stars'
            # Extraer sólo el número?
            rating = rating_text.split(" ")[0] if "out" in rating_text else "N/A"

            reviews_el = product.find("span", class_="a-size-base")
            reviews_count = reviews_el.get_text(strip=True) if reviews_el else "N/A"

            # Guardar en la lista
            all_products.append({
                "asin": asin,
                "title": title,
                "url": product_url,
                "price": price,
                "rating": rating,
                "reviews_count": reviews_count,
            })

        # Pequeña pausa para no saturar Amazon
        time.sleep(3)

    return all_products


def main():
    """
    Punto de entrada:
    1. Scrapea 1 o 2 páginas de Amazon con la KEYWORD.
    2. Guarda en CSV.
    """
    print(f"[INFO] Iniciando scraping en Amazon con la keyword: '{KEYWORD}'")

    products_data = scrape_amazon_competitors(KEYWORD, pages=2)  # Cambia 'pages' según te interese
    if not products_data:
        print("[WARNING] No se encontraron productos. Revisa si hubo algún bloqueo o si la página cambió.")
        return

    # Guardar en CSV
    print(f"[INFO] Guardando datos en '{OUTPUT_FILE}'")
    fieldnames = ["asin", "title", "url", "price", "rating", "reviews_count"]
    with open(OUTPUT_FILE, mode="w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for prod in products_data:
            writer.writerow(prod)

    print("[INFO] Proceso completado. Revisa el archivo CSV para los resultados.")


if __name__ == "__main__":
    main()

[INFO] Iniciando scraping en Amazon con la keyword: 'shopping'

[INFO] Scrape de la página 1 para la keyword 'shopping'
[WARNING] Status code 503 en la página 1. Deteniendo.
[WARNING] No se encontraron productos. Revisa si hubo algún bloqueo o si la página cambió.
